In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Training using Private Service Connect interface

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/jbrache/vertex-ai-things/blob/main/MLOps/Training/Vertex_Training_PSC_I.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fjbrache%2Fvertex-ai-things%2Fmain%2FMLOps%2FTraining%2FVertex_Training_PSC_I.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/jbrache/vertex-ai-things/blob/main/MLOps/Training/Vertex_Training_PSC_I.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/jbrache/vertex-ai-things/blob/main/MLOps/Training/Vertex_Training_PSC_I.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| Author(s) |
| --- |
| [Jose Brache](https://github.com/jbrache) |

## Overview

Private Service Connect interface is recommended for private connectivity since it reduces the chance of IP exhaustion and allows for transitive peering. See [Set up a Private Service Connect interface](https://cloud.google.com/vertex-ai/docs/general/vpc-psc-i-setup) on how to configure it for Vertex AI.

Review the public docs for the latest information on support: [Schedule training jobs based on resource availability](https://cloud.google.com/vertex-ai/docs/training/psc-i-egress)

This example covers the following steps:
1. Setup projects as the **VPC Host Project (Project 'a')** and the **Vertex AI Service Project (Project 'b')**
2. Create Test Training Code Container
3. Build/Push Custom Docker Container using Cloud Build
4. Use previously setup (from terrform) PSC-I network attachment in VPC Host Project for Vertex AI Training in **us-central1** or **us-west1**
6. Prepare Training Job
7. Submit Training Job with PSC-I in **us-central1**
8. Submit Training Job with PSC-I in **us-west1**
7. Clean Up

## Get started

### Install Vertex AI SDK and other required packages

In [ ]:
# import sys

# if "google.colab" in sys.modules:
#     !pip3 install --upgrade google-cloud-aiplatform

#     # Restart the notebook kernel after installs.
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

---

#### ⚠️ Do not forget to click the "RESTART RUNTIME" button above.

---

If you install additional packages, it's suggested to restart the notebook kernel so it can find the packages.

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

# 0-0. Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [23]:
############## Set Region for Shared Resources ###########################
REGION = "us-central1"  # @param {type: "string"}
# This is the region where the Vertex AI Test Container will be deployed
IMAGE_REGION = "us"  # @param {type: "string"}

############## Set VPC Host Project ###########################
# Calling this Project 'a' where the VPC Host project exists
PROJECT_ID_VPC_HOST = "jb01-shared-vpc-dev"  # @param {type:"string"}
VPC_NAME = "vertex-vpc-dev"

# Region #1
REGION_CENTRAL = "us-central1"  # @param {type: "string"}
SUBNET_NAME_CENTRAL = f"{REGION_CENTRAL}-vertex-psci"
NETWORK_ATTACHMENT_NAME_CENTRAL = f"{REGION_CENTRAL}-vertex-psci"

# Region #2
REGION_WEST = "us-west1"  # @param {type: "string"}
SUBNET_NAME_WEST = f"{REGION_WEST}-vertex-psci"
NETWORK_ATTACHMENT_NAME_WEST = f"{REGION_WEST}-vertex-psci"

############## Set Vertex AI Project ###########################
# Calling this Project 'b' where Vertex AI Training jobs run
PROJECT_ID = "ds-dev-jb0001"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
INFORMATION: Project 'ds-dev-jb0001' has no 'environment' tag set. Use either 'Production', 'Development', 'Test', or 'Staging'. Add an 'environment' tag using `gcloud resource-manager tags bindings create`.
Updated property [core/project].


## 0-1. Enable APIs
The following APIs are enabled in this demo:
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
2. [Enable the Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)
3. [Enable the Artifact Registry API](https://console.cloud.google.com/flows/enableapi?apiid=artifactregistry.googleapis.com): You must enable the Artifact Registry API for your project. You will store your custom training container in Artifact Registry. [Learn more about Enabling the Artifact Registry service](https://cloud.google.com/artifact-registry/docs/enable-service)


In [5]:
############# Enable the APIs for Vertex AI Project ########################
!gcloud services enable --project=$PROJECT_ID aiplatform.googleapis.com artifactregistry.googleapis.com cloudbuild.googleapis.com --project $PROJECT_ID

Operation "operations/acf.p2-277369590897-4f19a595-79d3-4bb1-b948-ed243c1517fb" finished successfully.


## 0-1. IAM Bindings

In [5]:
PROJECT_NUMBER = !(gcloud projects describe $PROJECT_ID --format="value(projectNumber)")
PROJECT_NUMBER = PROJECT_NUMBER[0]

In [6]:
COMPUTE_ENGINE_SERVICE_AGENT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:$COMPUTE_ENGINE_SERVICE_AGENT"  \
  --role="roles/cloudbuild.builds.builder"

Updated IAM policy for project [ds-dev-jb0001].
bindings:
- members:
  - serviceAccount:service-277369590897@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-277369590897@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:277369590897-compute@developer.gserviceaccount.com
  - serviceAccount:277369590897@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-277369590897@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-277369590897@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-277369590897@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - serviceAccount:277369590897@cloudservices.gserviceaccount.com
  role: roles

## 0-2. Import libraries

In [2]:
import os
import sys
import requests
import json

## 0-3. Training Code Container

In [12]:
# Training code container def
CONTAINER_DIR = "test_container"

Verify the location where the training code exists

In [9]:
# Remove if there's any such folder already
!rm -rf $CONTAINER_DIR
# Create your app directory
!mkdir -p $CONTAINER_DIR/trainer
# Create a subdirectory for store the training scripts
!touch $CONTAINER_DIR/trainer/train.py
# Create the init file
!touch $CONTAINER_DIR/trainer/__init__.py

In [10]:
# Print the location where the training code exists, will be used to create the artifact registry container
print(f"Local container directory with training code: {CONTAINER_DIR}")
print("Check whether the container directory exists:", os.path.exists(CONTAINER_DIR))

Local container directory with training code: test_container
Check whether the container directory exists: True


In [11]:
%%writefile $CONTAINER_DIR/trainer/train.py
import argparse
import logging
import sys
import os
import time
import json

def parse_args():
  """Parses command-line arguments."""
  """Argument parser.

  Returns:
    Dictionary of arguments.
  """
  parser = argparse.ArgumentParser()

  parser.add_argument('--log-level', help='Logging level.', choices=['DEBUG', 'ERROR', 'FATAL', 'INFO', 'WARN'], default='INFO')
  parser.add_argument('--sleep', help='Amount of time in seconds to sleep.', type=str, default='600s')
  parsed, unknown = parser.parse_known_args()
  return parsed, unknown

if __name__ == '__main__':
  """Entry point"""
  arguments, unknown_args = parse_args()
  logging.basicConfig(level=arguments.log_level)

  if arguments.sleep[-1] == "s":
    sleep = int(arguments.sleep[:-1])
  else:
    sleep = int(arguments.sleep)

  # Sleeping 600 seconds to connect the web shell
  logging.info(f'Sleeping for {sleep} seconds...')
  time.sleep(sleep)

Overwriting test_container/trainer/train.py


In [12]:
%%writefile $CONTAINER_DIR/requirements.txt

UsageError: %%writefile is a cell magic, but the cell body is empty.


In [13]:
%%writefile $CONTAINER_DIR/pyproject.toml
[tool.poetry]
package-mode = false

[tool.poetry.dependencies]
python = "==3.10.12"

Writing test_container/pyproject.toml


In [14]:
%%writefile $CONTAINER_DIR/Dockerfile
# https://medium.com/@albertazzir/blazing-fast-python-docker-builds-with-poetry-a78a66f5aed0

# Fetch the base image
# https://hub.docker.com/r/amd64/python/
# FROM amd64/python:3.10.12
# The builder image, used to build the virtual environment
FROM python:3.10.12-slim-bookworm

# Install pipx
RUN apt-get update && \
    apt-get install --no-install-suggests --no-install-recommends --yes pipx

# Install network tools: ping, dig, nslookup
RUN apt-get update && \
    apt-get install -y net-tools iputils-ping tcpdump dnsutils

ENV PATH="/root/.local/bin:${PATH}"
RUN pipx install poetry
RUN pipx inject poetry poetry-plugin-bundle

ENV POETRY_NO_INTERACTION=1 \
    POETRY_VIRTUALENVS_IN_PROJECT=1 \
    POETRY_VIRTUALENVS_CREATE=1 \
    POETRY_CACHE_DIR=/tmp/poetry_cache

# Set the working dir for the rest of the commands
WORKDIR /

# COPY requirements.txt .
COPY pyproject.toml .
# COPY poetry.lock .

RUN poetry install --no-root && rm -rf $POETRY_CACHE_DIR

ENV VIRTUAL_ENV=/.venv \
    PATH="/.venv/bin:$PATH"

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT [ "poetry", "run", "python", "-m", "trainer.train" ]

Writing test_container/Dockerfile


# 1-0. Build and Push Custom Container to Artifact Registry

You must have enabled the Artifact Registry API for your project in the previous steps. You will store your custom training container in Artifact Registry.

## 1-1. Create a private Docker repository
Your first step is to create a Docker repository in Artifact Registry.

1 - Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description `Docker repository`.

2 - Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
# Repo to create / use for running training job
PRIVATE_REPO = "vertex-training-test"
TRAIN_IMAGE = (
    f"{IMAGE_REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/test:latest"
)
print("Private Repo:", PRIVATE_REPO)
print("Training Container Image:", TRAIN_IMAGE)

Private Repo: test-training
Training Container Image: us-central1-docker.pkg.dev/ds-dev-jb0001/test-training/test:latest


In [16]:
!gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --project={PROJECT_ID} --location={REGION} --description="Docker repository"

Create request issued for: [test-training]
Waiting for operation [projects/ds-dev-jb0001/locations/us-central1/operations/
89512de1-7f6d-4e7c-8154-d3e0a2bced37] to complete...done.                      
Created repository [test-training].


In [11]:
!gcloud artifacts repositories --project={PROJECT_ID} list

Listing items under project ds-dev-jb0001, across all locations.

                                                                     ARTIFACT_REGISTRY
REPOSITORY     FORMAT  MODE                 DESCRIPTION        LOCATION     LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
test-training  DOCKER  STANDARD_REPOSITORY  Docker repository  us-central1          Google-managed key  2025-11-03T20:57:28  2025-11-03T20:57:28  0


## 1-2. Build and push the custom docker container image by using Cloud Build

Build and push a Docker image with Cloud Build

In [ ]:
!cd $CONTAINER_DIR && gcloud builds submit --timeout=1800s --project={PROJECT_ID} --region={IMAGE_REGION} --tag {TRAIN_IMAGE}

Creating temporary archive of 4 file(s) totalling 2.2 KiB before compression.
Uploading tarball of [.] to [gs://ds-dev-jb0001_cloudbuild/source/1762204886.607684-62de2d5e3ede428abd2913a816601f62.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/ds-dev-jb0001/locations/us-central1/builds/38e40945-d4f7-4593-80b6-e30aef41e7b3].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/38e40945-d4f7-4593-80b6-e30aef41e7b3?project=277369590897 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "38e40945-d4f7-4593-80b6-e30aef41e7b3"

FETCHSOURCE
Fetching storage object: gs://ds-dev-jb0001_cloudbuild/source/1762204886.607684-62de2d5e3ede428abd2913a816601f62.tgz#1762204887119271
Copying gs://ds-dev-jb0001_cloudbuild/source/1762204886.607684-62de2d5e3ede428abd2913a816601f62.tgz#1762204887119271...
/ [1 files][  1.5 KiB/  1.5 KiB]           

# 2-0. Set up private services access for your VPC

Following this guide for [setting up a Private Service Connect interface for Vertex AI resources](https://cloud.google.com/vertex-ai/docs/general/vpc-psc-i-setup).

In [14]:
PROJECT_NUMBER_VPC_HOST = !(gcloud projects describe $PROJECT_ID_VPC_HOST --format="value(projectNumber)")
PROJECT_NUMBER_VPC_HOST = PROJECT_NUMBER_VPC_HOST[0]

PROJECT_NUMBER = !(gcloud projects describe $PROJECT_ID --format="value(projectNumber)")
PROJECT_NUMBER = PROJECT_NUMBER[0]

In [15]:
print("PROJECT_ID:", PROJECT_ID)
print("PROJECT_NUMBER:", PROJECT_NUMBER)
print("----------")
print("PROJECT_ID_VPC_HOST:", PROJECT_ID_VPC_HOST)
print("PROJECT_NUMBER_VPC_HOST:", PROJECT_NUMBER_VPC_HOST)

PROJECT_ID: ds-dev-jb0001
PROJECT_NUMBER: 277369590897
----------
PROJECT_ID_VPC_HOST: jb01-shared-vpc-dev
PROJECT_NUMBER_VPC_HOST: 546029343527


In [16]:
FULL_NETWORK_ATTACHMENT_NAME_CENTRAL = f"projects/{PROJECT_NUMBER_VPC_HOST}/regions/{REGION_CENTRAL}/networkAttachments/{NETWORK_ATTACHMENT_NAME_CENTRAL}"
FULL_NETWORK_ATTACHMENT_NAME_WEST = f"projects/{PROJECT_NUMBER_VPC_HOST}/regions/{REGION_WEST}/networkAttachments/{NETWORK_ATTACHMENT_NAME_WEST}"

print(FULL_NETWORK_ATTACHMENT_NAME_CENTRAL)
print(FULL_NETWORK_ATTACHMENT_NAME_WEST)

projects/546029343527/regions/us-central1/networkAttachments/us-central1-vertex-psci
projects/546029343527/regions/us-west1/networkAttachments/us-west1-vertex-psci


## 2-3. List Subnets
These should have been created with terraform.

In [17]:
!gcloud compute networks subnets list --project={PROJECT_ID_VPC_HOST}

NAME                     REGION       NETWORK         RANGE        STACK_TYPE  IPV6_ACCESS_TYPE  INTERNAL_IPV6_PREFIX  EXTERNAL_IPV6_PREFIX  UTILIZATION_DETAILS
us-central1-vertex-psci  us-central1  vertex-vpc-dev  10.0.0.0/24  IPV4_ONLY


## 2-4. List Network Attachments

These should have been created with terraform.

In [18]:
!gcloud compute network-attachments list --project={PROJECT_ID_VPC_HOST}

NAME                     REGION       CONNECTION_PREFERENCE
us-central1-vertex-psci  us-central1  ACCEPT_AUTOMATIC


In [ ]:
# @title [Optional]: Setup your NFS instance
# !gcloud config set filestore/zone {ZONE}

# !gcloud filestore instances create {INSTANCE_NAME} \
#     --tier=TIER \
#     --file-share=name="file_share_name",capacity={SIZE} \
#     --network=name={NETWORK}

# 3-0. Run custom training jobs with PSC-I

This section creates a custom training job with a Private Service Connect interface. View the [documentation page](https://cloud.google.com/vertex-ai/docs/training/psc-i-egress) for the most up to date information.

## 3-1. Prepare training jobs

Vertex AI Training supports submiting custom training jobs with a prebuilt container, custom container and python application via **HTTP request, Vertex AI SDK or gcloud CLI**. Learn more [here](https://cloud.google.com/vertex-ai/docs/training/code-requirements).

In this example, we will demonstrate how to run a custom job with with custom containers. Please specify the images below to your custom images.
Note, if it's not a public image, please ensure it's already pushed to your project.

https://cloud.google.com/vertex-ai/docs/training/containers-overview

In [3]:
# @title Function Defs
### Create CPU Test Training Job
def create_single_replica_cpu_job(project_id: str, region: str, network_peering: str = None, network_attachment: str = None, enable_web_access: bool = False):
  ############ Set Job Service Endpoint ################
  endpoint= f"{region}-aiplatform.googleapis.com"
  job_request_uri = f"https://{endpoint}/v1beta1/projects/{project_id}/locations/{region}/customJobs"

  bearer_token = !gcloud auth application-default print-access-token
  headers = {
      'Content-Type': 'application/json',
      'Authorization': 'Bearer {}'.format(bearer_token[0]),
  }

  print("Calling endpoint: {} to create custom training job".format(job_request_uri))
  cpu_single_job = {
    "display_name": "CPU Test Job",
    "job_spec": {
        "worker_pool_specs": [
          CPU_WORKER_SPEC,
        ],
        "service_account": TRAINING_SA,
    },
    "labels": {
        "network_type": "none"
    }
  }

  if network_attachment and (network_peering == None):
    psc_interface_config = {
        "network_attachment": network_attachment
    }
    labels = {
        "network_type": "psc-i"
    }
    cpu_single_job["display_name"] = "CPU Test Job PSC-I"
    cpu_single_job["job_spec"]["psc_interface_config"] = psc_interface_config
    cpu_single_job["labels"] = labels

  if (network_attachment == None) and network_peering:
    labels = {
        "network_type": "peering"
    }
    cpu_single_job["display_name"] = "CPU Test Job Peering"
    cpu_single_job["job_spec"]["network"] = network_peering
    cpu_single_job["labels"] = labels

  if enable_web_access:
    cpu_single_job["job_spec"]["enable_web_access"] = enable_web_access

  print(json.dumps(cpu_single_job, indent=2))

  response = requests.post(job_request_uri, json=cpu_single_job, headers=headers)

  print("response:", response)
  job_name = None
  job_id = None
  if response.reason == 'OK':
    job_name = response.json()['name']
    job_id = job_name.split('/')[-1]
    url = f"console.cloud.google.com/vertex-ai/locations/{region}/training/{job_id}/cpu?&project={project_id}"
    print("Created Job: ", response.json()['name'])
    print("Link:", "https://"+url)
  else:
    print(response.text)
  return job_name, job_id, response

In [24]:
# Repo to create / use for running training job
PRIVATE_REPO = "vertex-training-test"
TRAIN_IMAGE = (
    f"{IMAGE_REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/test:latest"
)
print("Private Repo:", PRIVATE_REPO)
print("Training Container Image:", TRAIN_IMAGE)

Private Repo: vertex-training-test
Training Container Image: us-docker.pkg.dev/ds-dev-jb0001/vertex-training-test/test:latest


In [25]:
PROJECT_NUMBER_VPC_HOST = !(gcloud projects describe $PROJECT_ID_VPC_HOST --format="value(projectNumber)")
PROJECT_NUMBER_VPC_HOST = PROJECT_NUMBER_VPC_HOST[0]

PROJECT_NUMBER = !(gcloud projects describe $PROJECT_ID --format="value(projectNumber)")
PROJECT_NUMBER = PROJECT_NUMBER[0]

In [26]:
print("PROJECT_ID:", PROJECT_ID)
print("PROJECT_NUMBER:", PROJECT_NUMBER)
print("----------")
print("PROJECT_ID_VPC_HOST:", PROJECT_ID_VPC_HOST)
print("PROJECT_NUMBER_VPC_HOST:", PROJECT_NUMBER_VPC_HOST)

PROJECT_ID: ds-dev-jb0001
PROJECT_NUMBER: 277369590897
----------
PROJECT_ID_VPC_HOST: jb01-shared-vpc-dev
PROJECT_NUMBER_VPC_HOST: 546029343527


In [27]:
FULL_NETWORK_ATTACHMENT_NAME_CENTRAL = f"projects/{PROJECT_NUMBER_VPC_HOST}/regions/{REGION_CENTRAL}/networkAttachments/{NETWORK_ATTACHMENT_NAME_CENTRAL}"
FULL_NETWORK_ATTACHMENT_NAME_WEST = f"projects/{PROJECT_NUMBER_VPC_HOST}/regions/{REGION_WEST}/networkAttachments/{NETWORK_ATTACHMENT_NAME_WEST}"

print(FULL_NETWORK_ATTACHMENT_NAME_CENTRAL)
print(FULL_NETWORK_ATTACHMENT_NAME_WEST)

projects/546029343527/regions/us-central1/networkAttachments/us-central1-vertex-psci
projects/546029343527/regions/us-west1/networkAttachments/us-west1-vertex-psci


In [28]:
# Prepare training images
CPU_IMAGE = TRAIN_IMAGE
print("Using image: ", CPU_IMAGE)
print(f"Network Attachment in {REGION_CENTRAL}: {FULL_NETWORK_ATTACHMENT_NAME_CENTRAL}")
print(f"Network Attachment in {REGION_WEST}: {FULL_NETWORK_ATTACHMENT_NAME_WEST}")

Using image:  us-docker.pkg.dev/ds-dev-jb0001/vertex-training-test/test:latest
Network Attachment in us-central1: projects/546029343527/regions/us-central1/networkAttachments/us-central1-vertex-psci
Network Attachment in us-west1: projects/546029343527/regions/us-west1/networkAttachments/us-west1-vertex-psci


## 3-2. Training Job Config

In [29]:
CPU_MACHINE_TYPE = "n2-standard-4"  # @param {type:"string"}
TRAINING_SA = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

In [ ]:
# This is consistent with the default disk spec of jobs.
DISK_SPEC = {
  "boot_disk_type": "pd-ssd",
  "boot_disk_size_gb": 100
}

############ Set WorkerPool Spec #####################
# https://cloud.google.com/vertex-ai/docs/reference/rest/v1/CustomJobSpec
CPU_WORKER_SPEC = {
  "machine_spec": {
    "machine_type": CPU_MACHINE_TYPE,
  },
  "replica_count": 1,
  "container_spec": {
    "image_uri": CPU_IMAGE,
    "command": [],
    "args": [
        '--sleep=600s',
    ],
  },
  "disk_spec": DISK_SPEC
}

print('CPU worker spec:\n', json.dumps(CPU_WORKER_SPEC, indent=2))

CPU worker spec:
 {
  "machine_spec": {
    "machine_type": "n2-standard-4"
  },
  "replica_count": 1,
  "container_spec": {
    "image_uri": "us-docker.pkg.dev/ds-dev-jb0001/vertex-training-test/test:latest",
    "command": [],
    "args": [
      "--sleep=1200s"
    ]
  },
  "disk_spec": {
    "boot_disk_type": "pd-ssd",
    "boot_disk_size_gb": 100
  }
}


## 3-3. Create CPU test job on Vertex AI Training - us-central1

In [31]:
# us-central1 job
job_name_central, job_id_central, custom_job_response_central = create_single_replica_cpu_job(PROJECT_ID,
                                                                      REGION_CENTRAL,
                                                                      network_peering=None,
                                                                      network_attachment=FULL_NETWORK_ATTACHMENT_NAME_CENTRAL,
                                                                      enable_web_access=True)

Calling endpoint: https://us-central1-aiplatform.googleapis.com/v1beta1/projects/ds-dev-jb0001/locations/us-central1/customJobs to create custom training job
{
  "display_name": "CPU Test Job PSC-I",
  "job_spec": {
    "worker_pool_specs": [
      {
        "machine_spec": {
          "machine_type": "n2-standard-4"
        },
        "replica_count": 1,
        "container_spec": {
          "image_uri": "us-docker.pkg.dev/ds-dev-jb0001/vertex-training-test/test:latest",
          "command": [],
          "args": [
            "--sleep=1200s"
          ]
        },
        "disk_spec": {
          "boot_disk_type": "pd-ssd",
          "boot_disk_size_gb": 100
        }
      }
    ],
    "service_account": "277369590897-compute@developer.gserviceaccount.com",
    "psc_interface_config": {
      "network_attachment": "projects/546029343527/regions/us-central1/networkAttachments/us-central1-vertex-psci"
    },
    "enable_web_access": true
  },
  "labels": {
    "network_type": "psc-i"


In [12]:
print(json.dumps(json.loads(custom_job_response_central.text), indent=2))

{
  "error": {
    "code": 404,
    "message": "The Artifact Registry repository projects/ds-dev-jb0001/locations/us-central1/repositories/vertex-training-test does not exist.",
    "status": "NOT_FOUND",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.DebugInfo",
        "detail": "[ORIGINAL ERROR] generic::not_found: com.google.cloud.ai.platform.common.errors.AiPlatformException: code=NOT_FOUND, message=The Artifact Registry repository projects/ds-dev-jb0001/locations/us-central1/repositories/vertex-training-test does not exist., cause=null [google.rpc.error_details_ext] { code: 5 message: \"The Artifact Registry repository projects/ds-dev-jb0001/locations/us-central1/repositories/vertex-training-test does not exist.\" }"
      }
    ]
  }
}


## 3-4. Create CPU test job on Vertex AI Training - us-west1

In [ ]:
# us-west1 job
job_name_west, job_id_west, custom_job_response_west = create_single_replica_cpu_job(PROJECT_ID,
                                                                      REGION_WEST,
                                                                      network_peering=None,
                                                                      network_attachment=FULL_NETWORK_ATTACHMENT_NAME_WEST,
                                                                      enable_web_access=True)

In [ ]:
print(json.dumps(json.loads(custom_job_response_west.text), indent=2))

## 3-5. Monitor and debug training with an interactive shell

The jobs in this project have [enabled interactive shells](https://cloud.google.com/vertex-ai/docs/training/monitor-debug-interactive-shell) for the custom training resource. The interactive shell allows you to inspect the container where your training code is running.

You can navitage to the interactive shell with [these](https://cloud.google.com/vertex-ai/docs/training/monitor-debug-interactive-shell#navigate-console) instructions.

# 4-0. Get Job Details

In [ ]:
# Option 1: Use the Custom Job ID to get details
# JOB_ID = "" # @param {type:"string"}
# !gcloud beta ai custom-jobs describe {JOB_ID} --project={PROJECT_ID} --region={REGION}

In [ ]:
# Option 2: List existing custom jobs, filter running jobs and ones with the set label
# Lists the existing custom jobs, filters with the label set for these jobs
FILTER = '"(state!="JOB_STATE_SUCCEEDED" AND state!="JOB_STATE_FAILED" AND state!="JOB_STATE_CANCELLED") AND labels.network_type=psc-i"'
!gcloud beta ai custom-jobs list --project={PROJECT_ID} --region={REGION_CENTRAL} --filter={FILTER}

In [ ]:
!gcloud beta ai custom-jobs list --project={PROJECT_ID} --region={REGION_WEST} --filter={FILTER}

# 5-0. Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Artifacts Repository
- VPC Network

In [ ]:
# Set this to true only if you'd like to delete your bucket
# delete_bucket = False
delete_artifacts_repo = False

# No bucket used in this example
# if delete_bucket:
#   !gsutil rm -rf $JOB_DIR
#   !gsutil rm -r $BUCKET_URI

if delete_artifacts_repo:
  !gcloud artifacts repositories delete {PRIVATE_REPO} --project={PROJECT_ID} --location={REGION} --quiet